In [0]:
USE CATALOG crowd2fund;

CREATE OR REPLACE TEMP VIEW tv_add_withdraw_funds AS
SELECT Date, `Amount In (GBP)` as Amount
FROM bronze.v_transactions
WHERE `From/To` = 'Add Funds'
UNION ALL
SELECT Date, `Amount Out (GBP)` * -1 as Amount
FROM bronze.v_transactions
WHERE `From/To` = 'Withdrawal Request'
ORDER BY Date

In [0]:
SELECT * FROM tv_add_withdraw_funds ORDER BY date

In [0]:
%python
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

add_withdraw_funds = spark.table("tv_add_withdraw_funds")
window_spec = Window.orderBy("Date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
add_withdraw_funds = add_withdraw_funds.withColumn("Balance", sum("Amount").over(window_spec))
add_withdraw_funds.write.mode("overwrite").saveAsTable("crowd2fund.silver.add_withdraw_funds")

In [0]:
SELECT Date, Amount, Balance
FROM silver.add_withdraw_funds
ORDER BY Date

Databricks visualization. Run in Databricks to view.

In [0]:
SELECT SUM(Amount), COUNT(*)
FROM silver.add_withdraw_funds